In [1]:
#!/usr/bin/env python
# coding: utf-8

# Packages
import os
from os.path import join
import spacy
from nltk.text import Text
from termcolor import colored, cprint
from termcolor import colored
import json
import pandas as pd

# Load language model
nlp = spacy.load("da_core_news_sm")

## Set tokenizer
tokenizer = nlp.tokenizer

## Extended tokenizer
def tokenizer_ext(text, custom_stops = []):

    default_stopwords = list(nlp.Defaults.stop_words) # Indlæser prædefineret stopordsliste
    stop_words = default_stopwords + custom_stops # Danner samlet stopordsliste
    
    pos_tags = ['ADJ', 'NOUN'] # Definerer POS-tags som skal bevares

    doc = nlp(text)

    tokens = []

    for word in doc: # Looper igennem hvert ord i tweet
        if (len(word.lemma_) < 3): # Ord må ikke være mindre end 3 karakterer - går videre til næste ord, hvis det er
            continue
        if (word.pos_ in pos_tags) and (word.lemma_ not in stop_words): # Tjek at ordets POS-tag indgår i listen af accepterede tags og at ordet ikke er stopord
            tokens.append(word.lemma_) # Tilføj ordets lemma til tokens, hvis if-betingelse er opfyldt
                
    return(tokens)


# Data
data_p = join('/work', 'teaching-materials', 'data', 'dk_parl', 'dkparl_parsed_20221216.json')

with open(data_p, 'r', encoding='utf-8') as f:
    dkparl = json.load(f)


# Item texts as list
itemtexts = []
for doc in dkparl:
    items = []
    for k, item in doc.get('Items').items():
        items.append(item.get('ItemText'))

    itemtexts = itemtexts + items

## Select newest 1000 texts
texts_select = itemtexts[len(itemtexts)-1000:]

## Item texts as one string
texts_select_string = ''.join(texts_select)


In [2]:
## Print initial text
print(texts_select_string[0:10000])

Spm. nr. S 984
22) Til børne- og undervisningsministeren af:
Christoffer Aagaard Melson(V):
Når ministeren den 20. april 2020 udtalte til A4 Arbejdsliv, at »lige nu følger vi situationen med AMU-centrene nøje, så vi kan hjælpe de uddannelser, der har behov for det«, kan ministeren så præcisere, hvornår ministeren konkret mener at amu-centrene har »behov for« hjælp?
Værsgo.
Tak for det. Mit spørgsmål lyder som følger: Når ministeren den 20. april 2020 udtalte til A4 Arbejdsliv, at »lige nu følger vi situationen med AMU-centrene nøje, så vi kan hjælpe de uddannelser, der har behov for det«, kan ministeren så præcisere, hvornår ministeren konkret mener at amu-centrene har »behov for« hjælp?
Ministeren.
Nogenlunde omkring nu – det tror jeg vist er det tætteste, jeg kommer på et konkret svar. Jeg mener, at der er en udfordring og har også hørt de råb om hjælp, der kommer fra amu-sektoren. Når der var den indledning, altså faktisk grunden til, at jeg sagde det til at starte med, så er det, f

In [3]:
## Tokenize
text_tokens = [str(token) for token in tokenizer(texts_select_string)]

In [5]:
# Concordance
text_obj = Text(text_tokens)

## Print concordance
text_obj.concordance("indvandrere", width = 120)

Displaying 25 of 87 matches:
t jeg kan se , at det er kommuner med en høj andel af indvandrere og efterkommere af indvandrere , hvor smittetallene s
er med en høj andel af indvandrere og efterkommere af indvandrere , hvor smittetallene stiger og stiger . Jeg vil bare 
ren , at regeringens målsætning om at få 10.000 flere indvandrere i arbejde er lykkedes ? 
 Værsgo . 
 Tak for det . Je
ner , at regeringens målsætning om at få 10.000 flere indvandrere i arbejde er lykkedes ? 
 Ministeren . 
 Det er rigti
ingen har et ambitiøst mål om , at beskæftigelsen for indvandrere og efterkommere skal øges med 10.000 personer frem mo
 , at der er alt for mange med ikkevestlig baggrund , indvandrere og efterkommere , som rent faktisk står uden for arbe
det ganske kort : I 2008 havde vi 68.000 ikkevestlige indvandrere uden for arbejdsmarkedet , og det tal var i 2013 steg
 
 Mener ministeren , at beskæftigelsesfrekvensen for indvandrere og flygtninge i den arbejdsdygtige alder er høj nok ?
 
 Mener mi

In [10]:
## Concordance as list
conc_list = text_obj.concordance_list("indvandrere", width = 120, lines = 100)

In [11]:
## Top words
conc_string = ' '.join([conc.line for conc in conc_list])
conc_tokens = tokenizer_ext(conc_string, custom_stops=['indvandrer'])
top_tokens = list(pd.Series(conc_tokens).value_counts()[0:10].index)

# Highlight words - top 
words = top_tokens

highlighted_lines = []

for line in conc_list:
    line = line.line

    tokens = []
    for token in [str(token) for token in tokenizer(line)]:
        if any([word in token for word in words]):
            tokens.append(f"{colored(token, 'green')}")
        else:
            tokens.append(token)
    
    highl = ' '.join(tokens)
    highlighted_lines.append(highl)

for line in highlighted_lines[0:25]:
    print(line)


t jeg kan se , at det er kommuner med en høj andel af indvandrere og efterkommere af indvandrere , hvor smittetallene s
er med en høj andel af indvandrere og efterkommere af indvandrere , hvor smittetallene stiger og stiger . Jeg vil bare
ren , at regeringens målsætning om at få 10.000 flere indvandrere i arbejde er lykkedes ? 
  Værsgo . 
  Tak for det . Je
ner , at regeringens målsætning om at få 10.000 flere indvandrere i arbejde er lykkedes ? 
  Ministeren . 
  Det er rigti
ingen har et ambitiøst mål om , at beskæftigelsen for indvandrere og efterkommere skal øges med 10.000 personer frem mo
  , at der er alt for mange med ikkevestlig baggrund , indvandrere og efterkommere , som rent faktisk står uden for arbe
det ganske kort : I 2008 havde vi 68.000 ikkevestlige indvandrere uden for arbejdsmarkedet , og det tal var i 2013 steg
 
  Mener ministeren , at beskæftigelsesfrekvensen for indvandrere og flygtninge i den arbejdsdygtige alder er høj nok ?
 
  Mener ministeren , at beskæftig

In [30]:

## Highlight words - specific (dictionary approach)
words = ['arbejd', 'beskæftig']

highlighted_lines = []

for line in conc_list:
    line = line.line

    tokens = []
    for token in [str(token) for token in tokenizer(line)]:
        if any([word in token for word in words]):
            tokens.append(f"{colored(token, 'blue')}")
        else:
            tokens.append(token)
    
    highl = ' '.join(tokens)
    highlighted_lines.append(highl)

for line in highlighted_lines[0:25]:
    print(line.replace("\n", ""))

t jeg kan se , at det er kommuner med en høj andel af indvandrere og efterkommere af indvandrere , hvor smittetallene s
er med en høj andel af indvandrere og efterkommere af indvandrere , hvor smittetallene stiger og stiger . Jeg vil bare
ren , at regeringens målsætning om at få 10.000 flere indvandrere i arbejde er lykkedes ?   Værsgo .   Tak for det . Je
ner , at regeringens målsætning om at få 10.000 flere indvandrere i arbejde er lykkedes ?   Ministeren .   Det er rigti
ingen har et ambitiøst mål om , at beskæftigelsen for indvandrere og efterkommere skal øges med 10.000 personer frem mo
  , at der er alt for mange med ikkevestlig baggrund , indvandrere og efterkommere , som rent faktisk står uden for arbe
det ganske kort : I 2008 havde vi 68.000 ikkevestlige indvandrere uden for arbejdsmarkedet , og det tal var i 2013 steg
   Mener ministeren , at beskæftigelsesfrekvensen for indvandrere og flygtninge i den arbejdsdygtige alder er høj nok ?
   Mener ministeren , at beskæftigelsesf

In [25]:
# cluster

from sklearn.feature_extraction.text import CountVectorizer

conc_line_list = [conc.line for conc in conc_list]

vectorizer = CountVectorizer() # dan vectorizerfunktion
transformed_documents = vectorizer.fit_transform(conc_conc_list) # brug vectorizer på tekster

# Konverter fittet vectorizer til array
transformed_documents_as_array = transformed_documents.toarray()

# Konverter til data frame
count_df = pd.DataFrame(transformed_documents_as_array, columns = vectorizer.get_feature_names_out())

from sklearn.cluster import KMeans
kmeans = KMeans(3, random_state = 142)

identified_clusters = kmeans.fit_predict(count_df)
identified_clusters

array([1, 1, 2, 2, 0, 2, 0, 0, 0, 1, 2, 2, 0, 0, 0, 1, 0, 1, 1, 0, 1, 2,
       1, 2, 1, 2, 0, 2, 0, 0, 0, 2, 2, 0, 2, 2, 0, 0, 0, 2, 0, 1, 1, 0,
       1, 1, 2, 1, 1, 2, 2, 1, 0, 2, 0, 2, 2, 0, 1, 2, 1, 0, 0, 2, 2, 2,
       0, 0, 2, 1, 0, 0, 0, 0, 2, 2, 2, 1, 0, 0, 0, 0, 0, 2, 1, 2, 1],
      dtype=int32)

In [33]:
highlighted_lines = []

colourmap = {
    '0': 'green',
    '1': 'blue', 
    '2': 'red'
}

for line, cluster in dict(zip(conc_line_list, identified_clusters)).items():
    colour = colourmap.get(str(cluster))
    line_colour = f"{colored(line, colour)}"
    
    highlighted_lines.append(line_colour)
    
for line in highlighted_lines[0:30]:
    print(line.replace("\n", ""))    
    

t jeg kan se , at det er kommuner med en høj andel af indvandrere og efterkommere af indvandrere , hvor smittetallene s
er med en høj andel af indvandrere og efterkommere af indvandrere , hvor smittetallene stiger og stiger . Jeg vil bare 
ren , at regeringens målsætning om at få 10.000 flere indvandrere i arbejde er lykkedes ?  Værsgo .  Tak for det . Je
ner , at regeringens målsætning om at få 10.000 flere indvandrere i arbejde er lykkedes ?  Ministeren .  Det er rigti
ingen har et ambitiøst mål om , at beskæftigelsen for indvandrere og efterkommere skal øges med 10.000 personer frem mo
 , at der er alt for mange med ikkevestlig baggrund , indvandrere og efterkommere , som rent faktisk står uden for arbe
det ganske kort : I 2008 havde vi 68.000 ikkevestlige indvandrere uden for arbejdsmarkedet , og det tal var i 2013 steg
  Mener ministeren , at beskæftigelsesfrekvensen for indvandrere og flygtninge i den arbejdsdygtige alder er høj nok ?
le . Tidligere har det været sådan , at flygt